In [ ]:
import jtrace
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact
import ipywidgets as widgets
%matplotlib inline

In [ ]:
# Let's model my 12-inch f/5 Newtonian reflector.
# 12-inch is approx 0.3 m
# f/5 then means the focal length is 1.5 m
# Which then means the parabolic equation is:
# z = (x^2+y^2) / (4*1.5)

D = 0.3
fn = 5
f = D * fn
R = 2 * f
para = jtrace.Paraboloid(R, 0)
focal_plane = jtrace.Plane(f)
plate_scale = 1./f # radians / m

In [ ]:
def rays(theta_x, theta_y):
    # Point towards (0,0,0), but at an angle.  Need to determine pupil locations.
    rs = np.linspace(0, D/2, 10)
    # The above works if theta is 0.
    # If theta is not zero, then need to shift the rays depending on how far away their origins are.
    # We'll set the z-origin to be in the same plane as the focal plane.
    dx = f * np.tan(theta_x)
    dy = f * np.tan(theta_y)
    rays_ = []
    for r in rs:        
        phis = np.linspace(0, 2*np.pi, int(np.ceil(r/D*256)), endpoint=False)
        for phi in phis:
            rays_.append(
                jtrace.Ray(jtrace.Vec3(r*np.cos(phi)+dx, r*np.sin(phi)+dy, f),
                           jtrace.Vec3(-np.tan(theta_x), -np.tan(theta_y), -1),
                           0))
    return rays_

In [ ]:
@interact(theta_x=widgets.FloatSlider(min=-1,max=1,step=0.01,value=0.1),
          theta_y=widgets.FloatSlider(min=-1,max=1,step=0.01,value=0.1),
          focus=widgets.FloatSlider(min=-0.5, max=0.5, step=0.01,value=0.1))
def spot(theta_x, theta_y, focus):
    """Display a spot diagram for a Newtonian telescope.

    @param theta_x  Field angle in degrees
    @param theta_y  Field angle in degrees
    @param focus    Defocus distance in mm
    """
    focal_plane = jtrace.Plane(f+focus*1e-3)
    spots = []
    for ray in rays(theta_x*np.pi/180, theta_y*np.pi/180):
        isec = para.intersect(ray)
        rray = isec.reflectedRay(ray)
        isec = focal_plane.intersect(rray)
        spots.append([isec.x0, isec.y0])
    spots = np.array(spots)
    spots -= np.mean(spots, axis=0)
    spots *= plate_scale*206265 # meters -> arcseconds
    plt.figure(figsize=(4.5,4))
    plt.scatter(spots[:,0], spots[:,1], s=1, alpha=0.5)
    plt.xlim(-10, 10)
    plt.ylim(-10, 10)
    plt.title(r"$\theta_x = {:4.2f}\,,\theta_y = {:4.2f}\,, f={:4.2f}$".format(theta_x, theta_y, focus))
    plt.xlabel("arcseconds")
    plt.ylabel("arcseconds")